In [13]:
import os

from torchvision.io import read_image
from PIL import Image
from script_util import create_model_diffusion

from SDE_datareduction import get_test_data, get_data
from main import IMAGE_DATASET_PATH, STRUCTURE_DATASET_PATH, BASE_OUTPUT

from SDE_utils import *
from SDE_test import mae, count_parameters

In [32]:
# Load config and establish paths for testing an existing model
folder_path = r"C:\Users\tabor\Documents\Studie\Bachelor\Jaar 4\BEP\Results\Results_journal\SmallUNet_nblocks_2_noisesteps_250_smartsplit_False_3906"
model_path = os.path.join(folder_path, r"models\best_model.pth")
with open(os.path.join(folder_path, "config.json"), "r", encoding="utf-8") as f:
    config = json.load(f)

In [35]:
# Get dataloaders, model and diffusiontools
train_dataloader, val_dataloader, test_dataloader, train_dataset, val_dataset, test_dataset = get_data(image_dataset_path=IMAGE_DATASET_PATH,
                                                                      structure_dataset_path=STRUCTURE_DATASET_PATH, **config)

input_images, label_images, *_ = next(iter(test_dataloader))

model, diffusion = create_model_diffusion('cuda', **config)

Operating System: Windows. Number of DataLoader workers set to: 0
Loading dataset from: Images='C:\Users\tabor\Documents\Programming\MachineLearning\MachineLearningModels\data\figure_B_maxrange_5000\Output', Structures='C:\Users\tabor\Documents\Programming\MachineLearning\MachineLearningModels\data\figure_B_maxrange_5000\Structure'
Total dataset size: 5000 items.
Validation set size: 500
Using random split for train/test...
Train set size: 3500
Test set size: 1000


ValueError: Unsupported image size: 512

In [16]:
# Get the parameter count of the model and print
parameters = count_parameters(model)

print(parameters)

115181894


In [17]:
# Get a batch and sample images with this model
sample_count = 32 # Max value equal to value of batchsize
input_images, label_images, *_ = next(iter(test_dataloader))

model.load_state_dict(torch.load(model_path, weights_only=True))

samples = diffusion.p_sample_loop(model, sample_count, label_images)

03:32:39 - INFO: Sampling 32 images
2it [00:01,  1.36it/s]


KeyboardInterrupt: 

In [36]:
# Noise an image for a set amount of steps
image_path = r"C:\Users\tabor\Documents\Programming\MachineLearning\MachineLearningModels\data\ppt_image_noisy.png"
t = torch.tensor([50])
target_image, label_image, *_ = test_dataset[10]
x_start = target_image.to('cuda')
print(x_start.shape)
noisy_image, *_ = diffusion.noise_images(x_start, t)

noisy_image = tensor_to_PIL(noisy_image)
target_image = tensor_to_PIL(target_image.unsqueeze(0))
label_image = tensor_to_PIL(label_image.unsqueeze(0))

print(target_image[0].size)
noisy_image[0].save(os.path.join("results", "noisy_image.png"))
target_image[0].save(os.path.join("results","target_image.png"))
label_image[0].save(os.path.join("results", "label_image.png"))


torch.Size([3, 512, 512])
(512, 512)
